<a href="https://colab.research.google.com/github/YamunaKalindi/QRCodeProject/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The new, proper code starts here


In [ ]:
!pip install fasttext faiss-cpu onnx tf2onnx dask


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.6 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
ERROR: Operation cancelled by user
^C


In [ ]:
!pip install fasttext faiss-cpu onnx tf2onnx dask


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.8 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313475 sha256=08996ac8b5b79c600b8db89e24a088d3d53e16720dd3d5c3cef1ba11d0000986
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext
  Attempting uninstall: protobuf
    Foun

In [2]:
import dask.dataframe as dd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
from scipy.stats import entropy
from urllib.parse import urlparse
import re

# Enable mixed precision for faster training
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Load dataset using Dask
print("🔹 Loading dataset...")
data_path = '/content/cleaned_dataset.csv'
combined_df = dd.read_csv(data_path).compute()  # Convert to Pandas after loading
print(f"✅ Dataset loaded. Shape: {combined_df.shape}")

# Feature Extraction
def calculate_entropy(url):
    counts = Counter(url)
    total_chars = len(url)
    return -sum((count / total_chars) * np.log2(count / total_chars) for count in counts.values())

def subdomain_count(url):
    return urlparse(url).netloc.count('.')

def digit_ratio(url):
    return sum(c.isdigit() for c in url) / len(url)

def special_char_count(url):
    return sum(c in '@?&=' for c in url)

def contains_ip(url):
    return bool(re.search(r'\d+\.\d+\.\d+\.\d+', url))

print("🔹 Extracting features...")
combined_df['url_length'] = combined_df['URL'].apply(len)
combined_df['entropy'] = combined_df['URL'].apply(calculate_entropy)
combined_df['subdomain_count'] = combined_df['URL'].apply(subdomain_count)
combined_df['digit_ratio'] = combined_df['URL'].apply(digit_ratio)
combined_df['special_chars'] = combined_df['URL'].apply(special_char_count)
combined_df['contains_ip'] = combined_df['URL'].apply(contains_ip).astype(int)

print(f"✅ Features extracted. Sample:\n{combined_df[['url_length', 'entropy', 'subdomain_count', 'digit_ratio', 'special_chars', 'contains_ip']].head()}")

# Scale features
X = combined_df[['url_length', 'entropy', 'subdomain_count', 'digit_ratio', 'special_chars', 'contains_ip']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features scaled.")

# Define target variable
y = combined_df['label'].values

# Train-test split
print("🔹 Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print(f"✅ Dataset split. Train size: {X_train.shape}, Test size: {X_test.shape}")

# Optimize data loading
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build ANN Model with Batch Normalization & Dropout
print("🔹 Building ANN model...")
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, input_shape=(X_train.shape[1],), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("✅ Model compiled.")

# Train model with EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
print("🔹 Training model...")
history = model.fit(train_dataset, epochs=50, validation_data=test_dataset, callbacks=[early_stopping])
print("✅ Training complete.")

# Evaluate model
loss, accuracy = model.evaluate(test_dataset)
print(f"✅ Test Accuracy: {accuracy:.4f}")


🔹 Loading dataset...
✅ Dataset loaded. Shape: (235795, 2)
🔹 Extracting features...
✅ Features extracted. Sample:
   url_length   entropy  subdomain_count  digit_ratio  special_chars  \
0          32  3.929229                2          0.0              0   
1          24  3.970176                2          0.0              0   
2          30  4.164735                3          0.0              0   
3          27  4.060262                2          0.0              0   
4          34  3.917626                2          0.0              0   

   contains_ip  
0            0  
1            0  
2            0  
3            0  
4            0  
✅ Features scaled.
🔹 Splitting dataset...
✅ Dataset split. Train size: (188636, 6), Test size: (47159, 6)
🔹 Building ANN model...
✅ Model compiled.
🔹 Training model...
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2948/2948 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.8057 - loss: 0.5366 - val_accuracy: 0.8200 - val_loss: 0.4315
Epoch 2/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8166 - loss: 0.4352 - val_accuracy: 0.8216 - val_loss: 0.4267
Epoch 3/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8174 - loss: 0.4323 - val_accuracy: 0.8213 - val_loss: 0.4234
Epoch 4/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8171 - loss: 0.4302 - val_accuracy: 0.8224 - val_loss: 0.4215
Epoch 5/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8173 - loss: 0.4294 - val_accuracy: 0.8223 - val_loss: 0.4202
Epoch 6/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.8177 - loss: 0.4285 - val_accuracy: 0.8224 - val_loss: 0.4190
Epoch 7/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8175 - loss: 0.4288 - val_accuracy: 0.8230 - val_loss: 0.4174
Epoch 8/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8176 - loss: 0.4281 - val_a

In [3]:
from sklearn.metrics import classification_report

# Predict on test data
y_pred_probs = model.predict(X_test)  # Get probabilities
y_pred = (y_pred_probs > 0.5).astype(int)  # Convert to binary labels

# Print classification report
print("🔹 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))


1474/1474 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
🔹 Classification Report:
              precision    recall  f1-score   support

           0     0.9228    0.6406    0.7562     20124
           1     0.7821    0.9601    0.8620     27035

    accuracy                         0.8238     47159
   macro avg     0.8525    0.8004    0.8091     47159
weighted avg     0.8421    0.8238    0.8169     47159



In [6]:
import dask.dataframe as dd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, classification_report
from collections import Counter
from scipy.stats import entropy
from urllib.parse import urlparse
import re

# Enable mixed precision for faster training
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# Load dataset using Dask
print("🔹 Loading dataset...")
data_path = '/content/cleaned_dataset.csv'
combined_df = dd.read_csv(data_path).compute()  # Convert to Pandas after loading
print(f"✅ Dataset loaded. Shape: {combined_df.shape}")

# Feature Extraction
def calculate_entropy(url):
    counts = Counter(url)
    total_chars = len(url)
    return -sum((count / total_chars) * np.log2(count / total_chars) for count in counts.values())

def subdomain_count(url):
    return urlparse(url).netloc.count('.')

def digit_ratio(url):
    return sum(c.isdigit() for c in url) / len(url)

def special_char_count(url):
    return sum(c in '@?&=' for c in url)

def contains_ip(url):
    return bool(re.search(r'\d+\.\d+\.\d+\.\d+', url))

print("🔹 Extracting features...")
combined_df['url_length'] = combined_df['URL'].apply(len)
combined_df['entropy'] = combined_df['URL'].apply(calculate_entropy)
combined_df['subdomain_count'] = combined_df['URL'].apply(subdomain_count)
combined_df['digit_ratio'] = combined_df['URL'].apply(digit_ratio)
combined_df['special_chars'] = combined_df['URL'].apply(special_char_count)
combined_df['contains_ip'] = combined_df['URL'].apply(contains_ip).astype(int)

print(f"✅ Features extracted. Sample:\n{combined_df[['url_length', 'entropy', 'subdomain_count', 'digit_ratio', 'special_chars', 'contains_ip']].head()}")

# Scale features
X = combined_df[['url_length', 'entropy', 'subdomain_count', 'digit_ratio', 'special_chars', 'contains_ip']].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("✅ Features scaled.")

# Define target variable
y = combined_df['label'].values

# Compute class weights to balance dataset
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"🔹 Computed class weights: {class_weight_dict}")

# Train-test split
print("🔹 Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print(f"✅ Dataset split. Train size: {X_train.shape}, Test size: {X_test.shape}")

# Optimize data loading
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Build ANN Model with Batch Normalization & Dropout
print("🔹 Building ANN model...")
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, input_shape=(X_train.shape[1],), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),

    # Additional Hidden Layer
    tf.keras.layers.Dense(16, activation='relu'),

    tf.keras.layers.Dense(1, activation='sigmoid')
])


# Compile model with class weighting
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print("✅ Model compiled.")

# Train model with EarlyStopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
print("🔹 Training model...")
history = model.fit(train_dataset, epochs=50, validation_data=test_dataset, callbacks=[early_stopping], class_weight=class_weight_dict)
print("✅ Training complete.")

# Evaluate model
y_pred_prob = model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.7).astype(int)  # Adjust threshold to 0.7

# Compute ROC-AUC Score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"✅ ROC-AUC Score: {roc_auc:.4f}")

# Classification Report
print("🔹 Classification Report:")
print(classification_report(y_test, y_pred))

loss, accuracy = model.evaluate(test_dataset)
print(f"✅ Test Accuracy: {accuracy:.4f}")


🔹 Loading dataset...
✅ Dataset loaded. Shape: (235795, 2)
🔹 Extracting features...
✅ Features extracted. Sample:
   url_length   entropy  subdomain_count  digit_ratio  special_chars  \
0          32  3.929229                2          0.0              0   
1          24  3.970176                2          0.0              0   
2          30  4.164735                3          0.0              0   
3          27  4.060262                2          0.0              0   
4          34  3.917626                2          0.0              0   

   contains_ip  
0            0  
1            0  
2            0  
3            0  
4            0  
✅ Features scaled.
🔹 Computed class weights: {0: 1.1679379860319976, 1: 0.8742862439747868}
🔹 Splitting dataset...
✅ Dataset split. Train size: (188636, 6), Test size: (47159, 6)
🔹 Building ANN model...
✅ Model compiled.
🔹 Training model...
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2948/2948 ━━━━━━━━━━━━━━━━━━━━ 25s 6ms/step - accuracy: 0.7994 - loss: 0.6852 - val_accuracy: 0.8195 - val_loss: 0.4481
Epoch 2/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8155 - loss: 0.4607 - val_accuracy: 0.8221 - val_loss: 0.4378
Epoch 3/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8153 - loss: 0.4589 - val_accuracy: 0.8224 - val_loss: 0.4340
Epoch 4/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.8155 - loss: 0.4562 - val_accuracy: 0.8221 - val_loss: 0.4373
Epoch 5/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8148 - loss: 0.4550 - val_accuracy: 0.8234 - val_loss: 0.4348
Epoch 6/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8152 - loss: 0.4541 - val_accuracy: 0.8234 - val_loss: 0.4328
Epoch 7/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.8156 - loss: 0.4537 - val_accuracy: 0.8227 - val_loss: 0.4355
Epoch 8/50
2948/2948 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8159 - loss: 0.4524 - val

In [7]:
from tensorflow.keras.models import save_model

# Save the trained model
save_model(model, "qr_url_detector_model.h5")


In [8]:
model.save("qr_url_detector_model.keras")


To load the model later, use this code snippet:


from tensorflow.keras.models import load_model

model = load_model("qr_url_detector_model.keras")
